# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [15]:
from selenium import webdriver # criar o navegador
from selenium.webdriver.common.by import By # Localizar elementos (os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no teclado

navegador = webdriver.Chrome()

# Passo 1 - Entrar no google
navegador.get("https://www.google.com/")

# Passo 2 - Pesquisar a cotação do dólar
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 3 - Pegar a cotação do EURO
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 5 - Pegar a cotação do OURO
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)


5.0598
5.268213162
294.60


In [16]:
#!pip install selenium

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [17]:
# Passo 6 - Atualizar a minha base de dados com as novas cotações
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

- Atualizando os preços e o cálculo do Preço Final

In [18]:
# atualizar a cotação de acordo com a moeda correspondente
#dolar
#as linhas onde a coluna "moeda" = "Dolar"

tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)

#euro
tabela.loc[tabela["Moeda"]=="Euror", "Cotação"] = float(cotacao_euro)

#ouro
tabela.loc[tabela["Moeda"]=="Ouro", "Cotação"] = float(cotacao_ouro)

#alualizar preço de compra = preco original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#atualizar preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela) 

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0598,5059.749402,1.40,7083.649163
1,Carro Renault,4500.00,Euro,6.0000,27000.000000,2.00,54000.000000
2,Notebook Dell,899.99,Dólar,5.0598,4553.769402,1.70,7741.407983
3,IPhone,799.00,Dólar,5.0598,4042.780200,1.70,6872.726340
4,Carro Fiat,3000.00,Euro,6.0000,18000.000000,1.90,34200.000000
5,Celular Xiaomi,480.48,Dólar,5.0598,2431.132704,2.00,4862.265408
6,Joia 20g,20.00,Ouro,294.6000,5892.000000,1.15,6775.800000


### Agora vamos exportar a nova base de preços atualizada

In [19]:
tabela.to_excel("Produtos Novos.xlsx", index = False)
navegador.quit()